# NORTHWIND DATABASE

El **objetivo** de este ejercicio es generar una serie de consultas SQL que permitan extraer información relevante de la base de datos y luego exportar los resultados para construir dashboards o visualizaciones que proporcionen insights valiosos para la toma de decisiones empresariales.

La base de datos Northwind es una base de datos de ejemplo clásica que simula las operaciones de una empresa minorista. Contiene tablas relacionales que representan diferentes aspectos del negocio, como productos, clientes, pedidos y empleados. Utilizando consultas SQL, podrás explorar y analizar estos datos para comprender mejor el desempeño del negocio en diferentes áreas.

Una vez que hayas realizado las consultas SQL y obtenido los resultados, podrás utilizarlos para herramientas de visualización de datos como Tableau, Power BI, Streamlit o cualquier otra herramienta de tu elección. 

In [6]:
![imagen](./img/northwind-er-diagram.png) 

"[imagen]" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


Tendrás que elegir uno de los retos a resolver:

In [3]:
pip install ibis-framework[sqlite]

Note: you may need to restart the kernel to use updated packages.


In [47]:
import ibis 
import pandas as pd 
import matplotlib.pyplot as plt 

con = ibis.sqlite.connect('C:/Users/rodri/OneDrive/Escritorio/DATA_SCIENCE/CODIGO_CLASES/TheBridge_DSPT-1/2-Data_Analysis/6-Bases_de_datos/Práctica/data/northwind.db')

In [48]:
# Verificar la conexión listando las tablas en la base de datos
tables = con.list_tables()
#print("Tablas en la base de datos son:", tables)
tables 

['Alphabetical list of products',
 'Categories',
 'Category Sales for 1997',
 'Current Product List',
 'Customer and Suppliers by City',
 'CustomerCustomerDemo',
 'CustomerDemographics',
 'Customers',
 'EmployeeTerritories',
 'Employees',
 'Invoices',
 'Order Details',
 'Order Details Extended',
 'Order Subtotals',
 'Orders',
 'Orders Qry',
 'Product Sales for 1997',
 'ProductDetails_V',
 'Products',
 'Products Above Average Price',
 'Products by Category',
 'Quarterly Orders',
 'Regions',
 'Sales Totals by Amount',
 'Sales by Category',
 'Shippers',
 'Summary of Sales by Quarter',
 'Summary of Sales by Year',
 'Suppliers',
 'Territories',
 'sqlite_sequence']

In [8]:
#categories = con.table("Categories")
#products = con.table("Products") 

#joined_products = categories.join(products, "CategoryID")
#joined_products 

r0 := DatabaseTable: Categories
  CategoryID   int64
  CategoryName string
  Description  string
  Picture      binary

r1 := DatabaseTable: Products
  ProductID       !int64
  ProductName     !string
  SupplierID      int64
  CategoryID      int64
  QuantityPerUnit string
  UnitPrice       decimal
  UnitsInStock    int64
  UnitsOnOrder    int64
  ReorderLevel    int64
  Discontinued    !string

JoinChain[r0]
  JoinLink[inner, r1]
    r0.CategoryID == r1.CategoryID
  values:
    CategoryID:      r0.CategoryID
    CategoryName:    r0.CategoryName
    Description:     r0.Description
    Picture:         r0.Picture
    ProductID:       r1.ProductID
    ProductName:     r1.ProductName
    SupplierID:      r1.SupplierID
    QuantityPerUnit: r1.QuantityPerUnit
    UnitPrice:       r1.UnitPrice
    UnitsInStock:    r1.UnitsInStock
    UnitsOnOrder:    r1.UnitsOnOrder
    ReorderLevel:    r1.ReorderLevel
    Discontinued:    r1.Discontinued

In [17]:
#category_df = joined_products.select(["CategoryName", "ProductName"]).to_pandas()
#category_df.rename(columns={"ProductName": "Product"}, inplace=True)
#category_df.set_index("Product", inplace=True) 

### **1. Análisis de Ventas por Región**
Analiza las ventas de productos por región. El objetivo es identificar las regiones que generan más ingresos y las tendencias de ventas a lo largo del tiempo en esas regiones. 
Pista: Utiliza gráficos de barras y líneas para mostrar la evolución de las ventas, así como mapas geográficos para visualizar la distribución de ingresos por región. 

In [49]:
# Seleccionar las tablas necesarias
orders = con.table('Orders')
order_details = con.table('Order Details')
regions = con.table('Region')
territories = con.table('Territories')
employee_territories = con.table('EmployeeTerritories')

# Unir tablas para obtener la información de ventas por región
joined_data = (
    orders
    .join(order_details, orders.OrderID == order_details.OrderID)
    .join(employee_territories, orders.EmployeeID == employee_territories.EmployeeID)
    .join(territories, employee_territories.TerritoryID == territories.TerritoryID)
    .join(regions, territories.RegionID == regions.RegionID)
)

# Calcular las ventas por región y fecha (sin intentar convertir `datetime` en Ibis)
sales_by_region_date = (
    joined_data
    .group_by([regions.RegionDescription, orders.OrderDate]) 
    .aggregate(total_sales=(order_details.Quantity * order_details.UnitPrice).sum())
    .sort_by([regions.RegionDescription, orders.OrderDate])
)

# Ejecutar la consulta y convertir a pandas DataFrame
sales_by_region_date_df = sales_by_region_date.execute()

# Convertir la columna de fechas a formato datetime en pandas
sales_by_region_date_df['OrderDate'] = pd.to_datetime(sales_by_region_date_df['OrderDate'], errors='coerce')

# Eliminar filas con fechas no válidas
sales_by_region_date_df = sales_by_region_date_df.dropna(subset=['OrderDate'])

# Gráfico de líneas de las tendencias de ventas por región a lo largo del tiempo
plt.figure(figsize=(14, 8))
for region in sales_by_region_date_df['RegionDescription'].unique():
    region_data = sales_by_region_date_df[sales_by_region_date_df['RegionDescription'] == region]
    plt.plot(region_data['OrderDate'], region_data['total_sales'], label=region)

plt.xlabel('Fecha')
plt.ylabel('Ventas Totales')
plt.title('Tendencias de Ventas por Región a lo Largo del Tiempo')
plt.legend(title='Región')
plt.show()




KeyError: <Type.DATETIME: 'DATETIME'>

### **2. Análisis de Desempeño de Productos**
Analiza el desempeño de los productos de la empresa. Examina las ventas de productos por categoría, identificando los productos más vendidos y los menos vendidos. 

Pista: Utiliza gráficos de barras y de tarta para visualizar la participación de mercado de cada categoría y los ingresos generados por los productos individuales. 

### **3. Análisis de Rentabilidad de Clientes**
Analiza la rentabilidad de los clientes de la empresa. Examina los ingresos generados por cada cliente, así como los costos asociados con el servicio o producto proporcionado. 

Pista: Utiliza gráficos de dispersión y tablas dinámicas para identificar los clientes más rentables y aquellos que requieren mayor atención.

### **4. Análisis de Inventarios y Proveedores**
Analiza los niveles de inventario y la gestión de proveedores de la empresa. Examina el inventario disponible por categoría de productos, así como el rendimiento de los proveedores en términos de puntualidad de entrega y calidad de los productos. 

Pista: Utiliza gráficos de líneas y tablas dinámicas para mostrar tendencias y comparar el desempeño de los proveedores.

### **5. Análisis de Comportamiento de Empleados**
Analiza el comportamiento de los empleados de la empresa. Examina las ventas realizadas por cada empleado, así como su eficiencia en el manejo de pedidos y su puntualidad en la entrega de productos. 

Pista: Utiliza gráficos de barra y de dispersión para comparar el desempeño de los empleados y identificar áreas de mejora. 